# VGG

VGG出自论文:[Very Deep Convolutional NetWorks for Large-Scale Image Recognition](https://arxiv.org/pdf/1409.1556.pdf)

当然可以查看[VGG论文翻译(大部分)](https://blog.csdn.net/yuanlulu/article/details/84402709)

其在ILSVRC2014竞赛中定位任务是第一,分类任务是第二(第一是GoogleNet).可以说,VGG比GoogleNet更加方便,虽然分类任务不如GoogleNet,但是网络的构造却比GoogleNet要简单很多,所以VGG也比GoogleNet更加常用,之后很多模型也是依据VGG进行升级改造.

VGG可以通俗的说是AlexNet的升级,不过VGG比AlexNet更深,参数更多,其主要采用增加卷积层的方法来加深网络,结果发现深度越深,网络学习能力越好,分类能力越强.当深度增加到16-19层时,识别效果有较大提升,即VGG-16,VGG-19.网络虽然很大,但是厉害的是收敛速度会快于AlexNet.

接下来让我们来看看VGG的牛逼之处吧.

### 1.VGG 网络结构

首先让我们来看看VGG的网络结构

**conv(same),filters=$(3\times 3)$, stride=1;maxpool=$(2\times 2)$, stride=2**

![](../../../picture/259.png)

可以看出VGG含有多种不同的结构,其中使用最多的就是D与E也就是VGG-16和VGG-19,其两者的参数是138和144 millions.

虽然参数比较多,但是层级结构确是非常的清晰,首先以$(224\times 224)$作为输入,接着连续使用两次卷积(为什么这里使用两次卷积在后面会说明),然后接一个池化层,再来两次卷积层,接一个池化层,再来三个卷积层,池化层,三个卷积层,池化层,最后再来三个卷积层,池化层.可以看到结构非常的简单,而且是有规律可循,卷积核从64->128->512(也许作者认为512已经够多了,也就停止翻倍了).也就是如下所示:

![](../../../picture/260.png)

### 2. Receptive Fields

通过VGG架构,我们可以看出作者使用的卷积核全部是$(3\times 3)$的,作者是出于以下考虑:

在AlexNet中,第一层是使用$(11\times 11)$,stride 4,在OverFeat中使用的是$(7\times 7)$,stride 2;那么使用两个$(3\times 3)$的叠加即可代替一个$(5\times 5)$的感受野,或者使用三个叠加的$(3\times 3)$即可代替$(7\times 7)$的感受野.

那现在来解释一下这是为什么:

截取自Google的Inceptionv3 <Rethinking the Inception Architecture for Computer Vision>
    

![](../../../picture/261.png)

可以看到,如果使用$(3\times 3)$的卷积核连续进行两次卷积操作(从下到上),那么(从上到下)的感受野正好是相当于一个$(5\times 5)$的感受野.

实际上,感受野的计算是采用以下公式(**不需要考虑padding size**):

$r_{i} = s_{i} \cdot (r_{i+1} -1) + k_{i}$

其中$r_{i}$表示第$i$层输入的一个区域,$s_{i}$表示第${i}$层的步长,$k_{i}$表示第${i}$层卷积核的大小.

我们可以来尝试计算一下:



**Conv2:** $r_{2} = 1 \cdot (1 - 1) + 3 =3$

**Conv1:** $r_{1} = 1 \cdot (3 - 1) + 3 =5$

可以看到,映射出来的感受野的确是5.这样也就解释了为什么使用两个叠加的$(3\times 3)$的结果等同于使用一个$(5\times 5)$的感受野,同样使用三个叠加的$(3\times 3)$的结果等同于使用一个$(7\times 7)$的感受野.

那么这样做有什么好处呢?

> 1. $3\times 3$是最小能够捕获上下左右和中心的尺寸.

> 2. 两个$3\times 3$的卷基层的有限感受野是$5\times 5$;三个$3\times 3$的感受野是$7\times 7$,可以替代大的filter尺寸.

> 3. 多个$3\times 3$的卷基层比一个大尺寸filter卷基层有更多的非线性,使得判决函数更加具有判决性.

> 4. 多个$3\times 3$的卷积层比一个大的filter有更少的参数,假设卷积层的输入和输出的特征图的大小相同为C,那么三个$3\times 3$的卷积层参数个数为$3\times (3\times 3\times C^{2})=27C^{2}$, 然而一个$(7\times 7)$的卷积层参数为$49C^{2}$.

**Ps:**

在论文中还提到了$1\times 1$的卷积核,这可能有点疑惑,实际上对于$Channels=1$的时候确实没有多大的作用,但是随着$Channels$增加,那么做的就是一个通道上的线性操作.

### 3. 分类框架

现在我们先来看看分类框架,之后我们再来看看回归的部分.

#### 3.1 训练

**训练参数:**

VGG在训练过程上基本与AlexNet相似,不同的是其没有使用多裁剪输入(后面会说明),使用具有动量的梯度下降法来优化目标函数,训练的批大小是256,动量为0.9.训练过程中用到了L2正则化(L2惩罚因子为$5\cdot 10^{−4}$),在头两层全连接中用到了dropout正则化(dropout因子为0.5).初始学习率为$10^{-2}$,每当验证数据集上的准确率达到饱和状态时,学习率降低10倍,实际上在训练ImageNet的过程中学习率总共降低三次,共经过了370K次迭代(74轮).

跟AlexNet相比,尽管我们的网的参数数量更多,深度也更深,却需要更少的训练轮数就能收敛,我们推测可能是以下原因:

(a) 更深的深度和更小的卷积核引起的隐式正则化

(b) 对个别网络层进行预(训练)初始化

**网络初始化:**

初始化在深度学习中是非常重要的,因为深度网络中的梯度是不稳定了,不好的初始化为阻碍网络的工作,作者先使用表中的A配置结构来训练了一个浅的神经网络,使用随机初始化,使用了$W:\sigma(0, 10^{-2}), b:0$来训练,然后当我们训练更深的网络结构的时候,使用网络A中已训练好的参数初始化网络的前四个卷积层和后三个全连接层,剩余的中间层采用随机初始化.

值得注意的是,在作者提交论文后发现使用Glorot & Bengio(2010)的随机初始化过程,效果会更好.

**网络输入:**

VGG网络输入是固定大小$(224\times 224)$, 为了获得$(224\times 224)$大小的输入,VGG先将数据集进行缩放,缩放有两种方式:

1.我们将**缩放**图片的最小边定义为S,理论上S可以是任何不小于224的值:如果S = 224,训练图片的最小边长将被完整的裁剪下来;如果$S >> 224$,裁剪图像将对应于图像的一小部分,仅仅包含一个小对象的一部分.

2.对于S有两种选择:
> A: 一种是选定一个固定的S,也就是单尺度训练(single-scale training),一个是256,一个是384.

>B: 多尺度训练(multi-scale training),这种方案下每个图片都被独立的缩放和随机采样,缩放采用的S在一定范围$[Smin, Smax]$内随机变化(我们的Smin = 256,Smax = 512).由于图像中的目标可能具有不同的大小,因此这种方法对训练效果是有正向效果的.这也可以看做是通过**尺度抖动(scale jittering)**对训练集进行了增强,因为模型将被训练的能够在一系列不同尺寸上识别物体.

在获取到缩放后的图片之后,进行随机裁剪$(224\times 224)$(每张图片在每次epoch中被裁剪一次),为了进一步增大训练数据集,裁剪下来的图片会被随机水平翻转和RGB颜色变换(参考AlexNet).



#### 3.2 测试

在测试的时候,使用已经训练好的模型,以及输入图片(Q:测试尺度),测试尺度Q不一定非要等于训练尺度S,然后缩放后的图片会想在OverFeat一样的网络上运行,也就是FCN,最后得到的是在该测试尺度大小下的多个预测结果.

由于FCN网络可以应用在整张图片上,测试时就没必要再进行裁剪采样了(参考OverFeat),从而避免了为每个裁剪图片都重复计算的低效操作,作者将这种方法定义为密集(dense)评估.

但是与此同时,像GooGleNet,AlexNet,那样的多次裁剪采样确实可以改善模型的性能,与全卷积网络相比,这种采样方法得到的结果更好,作者将这种方法定义为多次裁剪(multi-crop)评估.

#### 3.3 评估

**数据集**

使用的是ILSVRC-2012数据集(ILSVRC-2012-2014三界挑战赛都是用的这个数据集).数据集包括1000个类别的图像,并分为三组:训练(130万张图像),验证(5万张图像)和测试(10万张图).使用两个指标评估分类的性能:top-1和top-5错误率.前者是多类分类误差,即被错误分类图像的比例;后者是ILSVRC中使用的主要评估标准,即图像真实类别在前5个预测类别之外的比例.

**单尺度评估**

根据网络配置在单个尺度上评估单个VGG模型的性能,测试的图像尺寸规则如下:对于固定的$S$,$Q = S$,尺度抖动的时候$S\in [Smin, Smax]$,那么$Q = 0.5 * (Smin + Smax)$.测试结果如下:

![](../../../picture/262.png)

首先我们看到,对于局部响应化(LRN)效果并没有多大提升,所以现在一般都很少使用LRN.

其次,我们观察到分类误差随着深度的增加而减小:从11层的网络A到19层的网络E测试结果的变化就能看出来,特别要注意的是,尽管深度相同.带三层$(1\times1)$卷积层的网络C要比带三层$(3\times3)$卷积层的网络D表现差.这表明,虽然附加的非线性确实有帮助(C比B好),使用卷积核捕获空间上下文也很重要(D比C好).当深度达到19层时,网络的错误率就会达到饱和(降不下去了),更深的模型可能要在更大的数据集上才能体现出优势.另外作者也尝试把网络B中的每一对$(3\times3)$卷积层换成一个$(5\times5)$的卷积层.替换之后的top-1错误率比网络B(测试时使用中心裁剪)高了7%,这表明使用小卷积核的层数更深的网络要优于使用大卷积核的层数更浅的网络.

最后,训练时采用尺度抖动(scale jittering)$(S \in [256; 512])比训练时固定尺度 $(S = 256 or S = 384)$获得的测试结果要好的多,就算测试时采用单尺度也一样.这证实了通过尺度抖动增强训练集确实有助于网络学习多尺度的图像统计特征.

**多尺度评估**

上面已经在单尺度上评估了模型的表现,现在我们一起来看看多尺度对测试结果的影响.也就是把同一张图片缩放成不同的尺寸大小(对应不同的Q值)在同一个网络上分别测试,对输出的结果求平均.由于**担心训练和测试的尺寸差异太大会引起性能下降**,使用固定尺度训练的模型评估的时候使用的3个测试尺度和训练尺度都很接近:$Q = {S − 32, S, S + 32}$.

同时,训练时的尺度抖动使得网络在测试时可以适应更大范围的尺度变化,所以针对训练时采用尺度抖动的模型$(S \in [Smin; Smax])$,评估时的3个尺度变化范围相对更大一些:$Q = {Smin, 0.5(Smin + Smax), Smax}$

![](../../../picture/263.png)

结果表明,多尺度评估可以获得更好的性能(跟Tabel3 中同样模型的的单尺度评估结果比).跟前面的结果类似,最深的两个网络(D和E)表现最好,尺度抖动训练出的模型要比单尺度训练的模型表现好.

**多次裁剪评估**

Table5中比较了密集评估(dense evaluation)和多次裁剪评估(mult-crop evaluation),作者还通过平均两种评估方案的soft-max输出来评估它们的互补性.下面可以看到,多次裁剪评估表现的比密集评估略微好一些,而且两种方法确实是互补的,因为它们的组合优于它们中的任何一种:

![](../../../picture/264.png)

**结论:**

在这项工作中,我们评估了用于大规模图像分类的深度卷积网络(最多19层).结果表明,网络的深度有利于分类精度,在传统的卷积网络架构(LeNet、AlexNet)基础上大幅增加网络层数就可以在ImageNet挑战赛的数据集上达到state-of-the-art的表现.我们的结果再次证实了深度在视觉领域的重要性.

### 4. 回归框架(定位)

#### 4.1 定位卷积网络

为了达到目标定位的目的,作者使用了一个非常深的卷积网络,它的最后一个全连接层可以预测边界框的位置.一个边界框由一个4-D向量表示,包括框的中心标,宽度和高度(也可以是左上右下的四个坐标).有两种预测边界框的方法可选:每次预测一个边界框,即所有类共享边界框预测网络(就像OverFeat-样
,single-class regression,SCR);或者每个类预测一个单独的边界框(per-classregression,PCR).第一个方法的最后一层输出一个4-D向量,后一个方法最后输出的是4000-D向量(因为数据集中有一千个类).除了边框预测层之外的部分我们使用的是网络结构D(VGG-16),这个网络有16层并且在分类任务中表现最好.

**训练:**

定位卷积网络的训练和分类非常像.最主要的区别是我们使用欧式距离作为损失函数,这个函数将会惩罚预测的边界框与真实值之间的偏差.作者分别在$S=256$和$S=384$的情况下训练了两个定位模型(实际上作者是想使用尺度抖动,但是没有时间),再使用对应(相同的训练尺度)的分类模型的参数对定位模型参数进行了初始化,初始学习率是$10^{−3}$.

**测试:**

作者考虑了两种测试方式:

>第一种只考虑边框,不考虑分类错误,边框是对图片进行中心裁剪后输入网络中预测得到的.

>第二种方法更加全面,使用密集型评估的方式在整张图片上预测,实际上和OverFeat很像,但是没有使用offset池化技术(说实话效果不是很如意),最后也是使用贪心合并算法(也就是把离的很近的边框融合在一起,对他们的坐标进行平均).

最后预测边框与真实边框的交并比(IOU)大于0.5才能认为是正确的预测.

**Reference:**

[1] [VGG 论文阅读记录](https://zhuanlan.zhihu.com/p/42233779)

[2] [CNN：接受视野(Receptive Field)](https://zhuanlan.zhihu.com/p/41955458)

[3] [这就是神经网络 2：VGG论文翻译(大部分)](https://blog.csdn.net/yuanlulu/article/details/84402709)